In [1]:
from basic_tools import *
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
h2_total=pd.read_csv(h2_total_par_filtered_path,index_col='phenotype')
h2_total.shape

(497, 7602)

In [8]:
data_merged=phenotypes_par_filtered.merge(h2_total[h2_total.columns[h2_total.columns.str.contains('uni')]],on='phenotype')
data_merged.columns=data_merged.columns.str.replace('uni.','')

In [9]:
#data_merged['description']=data_merged.apply(lambda x: '<a href="/phenotype/{}">'.format(x.name)+x['description']+'</a>',axis=1)
data_merged['UK Biobank Data Showcase Link']=data_merged['UK Biobank Data Showcase Link']#.map(lambda x: '<a href="{}">{}</a>'.format(x,x) if type(x)==str else '')
#=data_merged.apply(lambda x: '<a href="{}">'.format(x['UK Biobank Data Showcase Link'])+x['UK Biobank Data Showcase Link']+'</a>' if type(x),axis=1)

data_merged['-log10(h2_p)']=-np.log10(data_merged['Total h2 p']).round(2)
data_merged['prev(%)']=(data_merged['n_cases']/(data_merged['n_cases']+data_merged['n_controls'])*100).round(2)
data_merged['Mean Chi^2']=data_merged['Mean Chi^2'].round(2)
data_merged['Lambda GC']=data_merged['Lambda GC'].round(2)
data_merged['Intercept']=data_merged['Intercept'].round(2)

In [10]:
#data_merged['UK Biobank Data Showcase Link'].iloc[1]

In [45]:
data_merged.columns

Index(['PHESANT_transformation', 'Sex', 'description', 'n_cases', 'n_controls',
       'n_missing', 'n_non_missing', 'notes', 'source', 'variable_type',
       'Phenotype Code', 'UK Biobank Data Showcase Link', 'File',
       'wget command', 'Dropbox File', 'filename', 'Total Observed scale h2',
       'Total Observed scale h2_se', 'Total Liability scale h2',
       'Total Liability scale h2_se', 'Lambda GC', 'Mean Chi^2', 'Intercept',
       'Intercept_se', 'Ratio', 'Total h2 z', 'Total h2 p', '-log10(h2_p)',
       'prev(%)'],
      dtype='object')

In [46]:
data_merged=data_merged[['description', 'Sex', 
       'Total Observed scale h2', 'Total Observed scale h2_se','Total Liability scale h2', 'Total Liability scale h2_se','-log10(h2_p)',
        'Intercept', 'Intercept_se','Ratio','Lambda GC','Mean Chi^2',  
        'prev(%)','n_missing', 'n_non_missing', 'n_cases', 'n_controls',               
       'UK Biobank Data Showcase Link', 'File',# 'Dropbox File',
        'source', 'variable_type','PHESANT_transformation','notes'
        ]]

In [47]:
tosave=data_merged

In [13]:
h2_total.columns

Index(['bp300.filename', 'bp300.Total Observed scale h2',
       'bp300.Total Observed scale h2_se', 'bp300.Total Liability scale h2',
       'bp300.Total Liability scale h2_se', 'bp300.Lambda GC',
       'bp300.Mean Chi^2', 'bp300.Intercept', 'bp300.Intercept_se',
       'bp300.Ratio',
       ...
       'uni.Total Observed scale h2_se', 'uni.Total Liability scale h2',
       'uni.Total Liability scale h2_se', 'uni.Lambda GC', 'uni.Mean Chi^2',
       'uni.Intercept', 'uni.Intercept_se', 'uni.Ratio', 'uni.Total h2 z',
       'uni.Total h2 p'],
      dtype='object', length=7602)

In [14]:
data_merged.columns

Index(['PHESANT_transformation', 'Sex', 'description', 'n_cases', 'n_controls',
       'n_missing', 'n_non_missing', 'notes', 'source', 'variable_type',
       'Phenotype Code', 'UK Biobank Data Showcase Link', 'File',
       'wget command', 'Dropbox File', 'filename', 'Total Observed scale h2',
       'Total Observed scale h2_se', 'Total Liability scale h2',
       'Total Liability scale h2_se', 'Lambda GC', 'Mean Chi^2', 'Intercept',
       'Intercept_se', 'Ratio', 'Total h2 z', 'Total h2 p', '-log10(h2_p)',
       'prev(%)'],
      dtype='object')

In [19]:
data_merged.to_csv("phenotype_info.csv")

In [21]:
phenotypes_par_filtered.columns

Index(['PHESANT_transformation', 'Sex', 'description', 'n_cases', 'n_controls',
       'n_missing', 'n_non_missing', 'notes', 'source', 'variable_type',
       'Phenotype Code', 'UK Biobank Data Showcase Link', 'File',
       'wget command', 'Dropbox File'],
      dtype='object')

In [24]:
phenotypes_par_filtered[['PHESANT_transformation', 'Sex', 'description', 'n_cases', 'n_controls',
       'n_missing', 'n_non_missing', 'notes', 'source', 'variable_type',
       'Phenotype Code', 'UK Biobank Data Showcase Link', 'File',]].to_csv("phenotype_info.csv")